In [52]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split

In [53]:
def preprocess_images(folder, label, image_size=(128, 128)):
    data = []
    labels = []

    for img_name in os.listdir(folder):
        img_path = os.path.join(folder, img_name)
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = cv2.resize(img, image_size)
        img = img / 255.0  # Normalize pixel values to the range [0, 1]

        data.append(img)
        labels.append(label)

    return np.array(data), np.array(labels)

In [54]:
dataset_pos = 'datasets/brain_mri_scan_images/positive'
dataset_neg = 'datasets/brain_mri_scan_images/negative'

positive_data, positive_labels = preprocess_images(dataset_pos, 1)
negative_data, negative_labels = preprocess_images(dataset_neg, 0)

x = np.concatenate((positive_data, negative_data))
y = np.concatenate((positive_labels, negative_labels))

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)

x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

# print(x_train, x_test)
# print(y_train, y_test)

print(f'Train: X={x_train.shape}, y={y_train.shape}')
print(f'Test: X={x_test.shape}, y={y_test.shape}')

input_shape = x_train.shape[1:4]
print(input_shape)


Train: X=(194, 128, 128, 1), y=(194,)
Test: X=(22, 128, 128, 1), y=(22,)
(128, 128, 1)


In [55]:
from tensorflow import keras
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Model, load_model
from keras.layers import Conv2D, Conv2DTranspose, Dense, Flatten
from keras.layers.pooling import MaxPooling2D
from keras.layers import Input, concatenate
from keras.layers.core import Dropout, Lambda

inputs = Input((128, 128, 1))

s = Lambda(lambda x: x / 255) (inputs)

c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (s)
c1 = Dropout(0.1) (c1)
c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c1)
p1 = MaxPooling2D((2, 2)) (c1)

c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p1)
c2 = Dropout(0.1) (c2)
c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c2)
p2 = MaxPooling2D((2, 2)) (c2)

c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p2)
c3 = Dropout(0.2) (c3)
c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c3)
p3 = MaxPooling2D((2, 2)) (c3)

c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p3)
c4 = Dropout(0.2) (c4)
c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c4)
p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p4)
c5 = Dropout(0.3) (c5)
c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c5)

u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (c5)
u6 = concatenate([u6, c4])
c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u6)
c6 = Dropout(0.2) (c6)
c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c6)

u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c6)
u7 = concatenate([u7, c3])
c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u7)
c7 = Dropout(0.2) (c7)
c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c7)

u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c7)
u8 = concatenate([u8, c2])
c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u8)
c8 = Dropout(0.1) (c8)
c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c8)

u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c8)
u9 = concatenate([u9, c1], axis=3)
c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u9)
c9 = Dropout(0.1) (c9)
c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c9)

f1 = Flatten() (c9)

outputs = Dense(1, activation="softmax") (f1)

model = Model(inputs=[inputs], outputs=[outputs])

model.compile(optimizer='adam', loss='binary_crossentropy')



In [56]:
batch_size = 8
epochs = 15

filepath = "model.h5"

earlystopper = EarlyStopping(patience=5, verbose=1)

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min')

callbacks_list = [earlystopper, checkpoint]


model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_11 (InputLayer)          [(None, 128, 128, 1  0           []                               
                                )]                                                                
                                                                                                  
 lambda_9 (Lambda)              (None, 128, 128, 1)  0           ['input_11[0][0]']               
                                                                                                  
 conv2d_161 (Conv2D)            (None, 128, 128, 16  160         ['lambda_9[0][0]']               
                                )                                                                 
                                                                                            

In [57]:
batch_size = 15
epochs = 30

history = model.fit(x_train, y_train, validation_split=0.1, batch_size=batch_size, epochs=epochs, 
                    callbacks=callbacks_list)

Epoch 1/30
11/12 [==========================>...] - ETA: 0s - loss: 12.0687 - accuracy: 0.5879
Epoch 1: val_loss improved from inf to 3.53347, saving model to model.h5
12/12 [==============================] - 2s 68ms/step - loss: 12.6725 - accuracy: 0.6034 - val_loss: 3.5335 - val_accuracy: 0.4500
Epoch 2/30
11/12 [==========================>...] - ETA: 0s - loss: 10.7709 - accuracy: 0.6242
Epoch 2: val_loss did not improve from 3.53347
12/12 [==============================] - 0s 39ms/step - loss: 10.3212 - accuracy: 0.6034 - val_loss: 9.6944 - val_accuracy: 0.4500
Epoch 3/30
11/12 [==========================>...] - ETA: 0s - loss: 14.4582 - accuracy: 0.6000
Epoch 3: val_loss did not improve from 3.53347
12/12 [==============================] - 0s 39ms/step - loss: 14.4228 - accuracy: 0.6034 - val_loss: 6.2187 - val_accuracy: 0.4500
Epoch 4/30
11/12 [==========================>...] - ETA: 0s - loss: 4.5597 - accuracy: 0.6061
Epoch 4: val_loss did not improve from 3.53347
12/12 [=======